In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [64]:
from importlib import reload
datasets = reload(datasets)

In [55]:
from rdforecast import datasets
data = datasets.load_training_data()

'filepath' not given, download data from: https://s3-ap-southeast-1.amazonaws.com/grab-aiforsea-dataset/traffic-management.zip
Data loaded.
N: 4206321
  geohash6  day timestamp    demand
0   qp03wc   18      20:0  0.020072
1   qp03pn   10     14:30  0.024721
2   qp09sw    9      6:15  0.102821


In [58]:
df = check_sanity(data)

No missing values found.
First day in sequence: 1
Last day in sequence: 61


In [61]:
train, test = split_train_test(df, path='./input/')

Train data size: 3213492 (47 days)
Shape: (3213492, 4)

Test data size: 992829 (14 days)
Shape: (992829, 4)

Saving split datasets to ./input/
Done.


In [80]:
time_parser = lambda x: pd.datetime.strptime(x, '%H:%M').time()
tmp = pd.read_csv('./input/train.csv', parse_dates=['timestamp'], date_parser=time_parser)

In [91]:
def get_time_features(df):
    h, m = zip(*[t.split(':') for t in df['timestamp']])
    h = np.array([int(i) for i in h])
    m = np.array([int(i) for i in m])
    df['timestep'] = ((h * 60 + m) / 15) + (df['day'].values - 1) * 96
    df['hour'] = h
    df['dow'] = df['day'] % 7
    df['weekly'] = df['timestep'] % 672
    return df

In [92]:
train = get_time_features(train)

/Users/Yuwen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/Yuwen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/Yuwen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

In [93]:
train.head()

,geohash6,day,timestamp,demand,time,timestep,hour,dow,weekly
0,qp03wc,18,20:0,0.020072,20:00:00,1712.0,20,4,368.0
1,qp03pn,10,14:30,0.024721,14:30:00,922.0,14,3,250.0
2,qp09sw,9,6:15,0.102821,06:15:00,793.0,6,2,121.0
3,qp0991,32,5:0,0.088755,05:00:00,2996.0,5,4,308.0
4,qp090q,15,4:0,0.074468,04:00:00,1360.0,4,1,16.0


In [69]:
train.head()

,geohash6,day,timestamp,demand,time
0,qp03wc,18,20:0,0.020072,20:00:00
1,qp03pn,10,14:30,0.024721,14:30:00
2,qp09sw,9,6:15,0.102821,06:15:00
3,qp0991,32,5:0,0.088755,05:00:00
4,qp090q,15,4:0,0.074468,04:00:00


In [66]:
train = datasets.convert_datetime(train)
test = datasets.convert_datetime(test)

Converting datetime features...


/Users/Yuwen/Dropbox/pj_RideForecast/rdforecast/datasets.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['time'] = df['timestamp'].apply(lambda x: datetime.strptime(x, '%H:%M').time())


Done.
Converting datetime features...
Done.


In [43]:
df = data.copy(deep=True)

In [26]:
train = datasets.convert_datetime(train)

In [ ]:
taz_activity = df['geohash6'].value_counts().values()
demands = df['demand'].values()

def plot_data_dist(df):


In [27]:
data.head()

,geohash6,day,timestamp,demand,date,time,datetime
0,qp03wc,18,20:0,0.020072,2018-01-19,20:00:00,2018-01-19 20:00:00
1,qp03pn,10,14:30,0.024721,2018-01-11,14:30:00,2018-01-11 14:30:00
2,qp09sw,9,6:15,0.102821,2018-01-10,06:15:00,2018-01-10 06:15:00
3,qp0991,32,5:0,0.088755,2018-02-02,05:00:00,2018-02-02 05:00:00
4,qp090q,15,4:0,0.074468,2018-01-16,04:00:00,2018-01-16 04:00:00
